# Comparaison des Stratégies de Portefeuille Obligataire

Ce notebook illustre l'utilisation du projet "Bond Portfolio Strategy" pour comparer les performances de différentes stratégies de portefeuille obligataire.

Nous allons explorer les trois approches principales :
- **Laddering** : Répartition échelonnée des obligations sur différentes maturités
- **Barbell** : Concentration sur les échéances courtes et longues, en évitant les échéances intermédiaires
- **Bullet** : Concentration des investissements autour d'une échéance cible

Nous analyserons également l'impact des cycles économiques et des régimes de taux d'intérêt sur ces stratégies.

## 1. Configuration et Chargement des Données

In [ ]:
import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

# Ajouter le répertoire parent au chemin pour pouvoir importer les modules du projet
sys.path.append('..')

# Importer les modules du projet
from data.data_loader import DataLoader
from models.laddering_strategy import LadderingStrategy
from models.barbell_strategy import BarbellStrategy
from models.bullet_strategy import BulletStrategy
from backtesting.backtest_engine import BacktestEngine
from analysis.economic_cycle_analyzer import EconomicCycleAnalyzer
from visualization.performance_visualizer import PerformanceVisualizer

# Configuration matplotlib
plt.style.use('seaborn')
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['font.size'] = 12

In [ ]:
# Définir les paramètres du backtest
start_date = '2010-01-01'
end_date = '2024-12-31'
rebalance_frequency = 'M'  # Mensuel
initial_capital = 1000000  # 1 million de dollars
benchmark = 'AGG'  # iShares Core U.S. Aggregate Bond ETF

# Si vous avez une clé API FRED, vous pouvez la définir ici
# Sinon, laissez api_key = None
api_key = None  # Remplacer par votre clé API FRED si disponible

# Initialiser le chargeur de données
data_loader = DataLoader(api_key=api_key)

# Charger les données obligataires
print("Chargement des données obligataires...")
etf_data = data_loader.load_bond_etfs(start_date=start_date, end_date=end_date)
treasury_yields = data_loader.load_treasury_yields(start_date=start_date, end_date=end_date)

# Afficher les ETF disponibles
print(f"ETF disponibles: {etf_data.columns.tolist()}")

## 2. Examen des Données de Rendement des Obligations

In [ ]:
# Visualiser les rendements des bons du Trésor
plt.figure(figsize=(14, 8))
for col in treasury_yields.columns:
    plt.plot(treasury_yields.index, treasury_yields[col], label=col)

plt.title('Rendements des Bons du Trésor Américain')
plt.xlabel('Date')
plt.ylabel('Rendement (%)')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Calculer les écarts de rendement (spreads)
if '10Y' in treasury_yields.columns and '3M' in treasury_yields.columns:
    treasury_yields['10Y_3M_Spread'] = treasury_yields['10Y'] - treasury_yields['3M']

if '10Y' in treasury_yields.columns and '2Y' in treasury_yields.columns:
    treasury_yields['10Y_2Y_Spread'] = treasury_yields['10Y'] - treasury_yields['2Y']

# Visualiser les spreads
plt.figure(figsize=(14, 6))
if '10Y_3M_Spread' in treasury_yields.columns:
    plt.plot(treasury_yields.index, treasury_yields['10Y_3M_Spread'], label='10Y-3M Spread')
    
if '10Y_2Y_Spread' in treasury_yields.columns:
    plt.plot(treasury_yields.index, treasury_yields['10Y_2Y_Spread'], label='10Y-2Y Spread')

plt.axhline(y=0, color='r', linestyle='-', alpha=0.3, label='Inversion de la courbe')
plt.title('Écarts de Rendement des Bons du Trésor')
plt.xlabel('Date')
plt.ylabel('Spread (%)')
plt.legend()
plt.grid(True)
plt.show()

## 3. Initialisation des Stratégies

In [ ]:
# Stratégie Laddering
laddering = LadderingStrategy(
    name="Laddering",
    maturities=[1, 2, 3, 5, 7, 10],  # Échéances en années
    initial_capital=initial_capital,
    rebalance_frequency=rebalance_frequency
)

# Stratégie Barbell
barbell = BarbellStrategy(
    name="Barbell",
    short_weight=0.5,  # 50% en obligations à court terme
    long_weight=0.5,   # 50% en obligations à long terme
    short_term_max=3,  # Définit les obligations à court terme comme celles <= 3 ans
    long_term_min=10,  # Définit les obligations à long terme comme celles >= 10 ans
    initial_capital=initial_capital,
    rebalance_frequency=rebalance_frequency
)

# Stratégie Bullet avec cible de 5 ans
bullet_5y = BulletStrategy(
    name="Bullet (5Y)",
    target_maturity=5,  # Cibler les obligations à 5 ans
    maturity_range=1,   # Plage de +/- 1 an autour de la cible
    initial_capital=initial_capital,
    rebalance_frequency=rebalance_frequency
)

# Stratégie Bullet avec cible de 10 ans
bullet_10y = BulletStrategy(
    name="Bullet (10Y)",
    target_maturity=10,  # Cibler les obligations à 10 ans
    maturity_range=2,    # Plage de +/- 2 ans autour de la cible
    initial_capital=initial_capital,
    rebalance_frequency=rebalance_frequency
)

## 4. Exécution du Backtest

In [ ]:
# Initialiser le moteur de backtest
backtest_engine = BacktestEngine(
    data_loader=data_loader,
    strategies=[laddering, barbell, bullet_5y, bullet_10y],
    benchmark_ticker=benchmark
)

# Exécuter le backtest
print("Exécution du backtest...")
results = backtest_engine.run_backtest(
    start_date=start_date,
    end_date=end_date,
    rebalance_frequency=rebalance_frequency
)

# Afficher le résumé des performances
performance_summary = backtest_engine.get_performance_summary()
performance_summary

## 5. Visualisation des Performances

In [ ]:
# Initialiser le visualiseur de performances
visualizer = PerformanceVisualizer()

# Extraire les données des portefeuilles
portfolios = {name: result['portfolio'] for name, result in results.items()}
benchmark_data = portfolios.pop(f"Benchmark ({benchmark})")

# Visualiser l'évolution de la valeur du portefeuille
fig1 = visualizer.plot_portfolio_value(portfolios, benchmark_data)
plt.show()

In [ ]:
# Visualiser les drawdowns
fig2 = visualizer.plot_drawdown(portfolios, benchmark_data)
plt.show()

In [ ]:
# Visualiser les rendements glissants sur 1 an
fig3 = visualizer.plot_rolling_metrics(portfolios, benchmark_data, metric='returns', window=252)
plt.show()

In [ ]:
# Visualiser la volatilité glissante sur 1 an
fig4 = visualizer.plot_rolling_metrics(portfolios, benchmark_data, metric='volatility', window=252)
plt.show()

In [ ]:
# Visualiser le ratio de Sharpe glissant sur 1 an
fig5 = visualizer.plot_rolling_metrics(portfolios, benchmark_data, metric='sharpe', window=252)
plt.show()

In [ ]:
# Visualiser les rendements mensuels
fig6 = visualizer.plot_returns_comparison(portfolios, benchmark_data, period='M')
plt.show()

## 6. Analyse des Cycles Économiques et Impact sur les Stratégies

In [ ]:
# Analyse des cycles économiques (nécessite une clé API FRED)
if api_key:
    cycle_analyzer = EconomicCycleAnalyzer(data_loader)
    
    # Charger les données économiques
    economic_data = cycle_analyzer.load_economic_data(
        start_date=start_date,
        end_date=end_date
    )
    
    # Identifier les cycles économiques
    cycles = cycle_analyzer.identify_cycles(method='composite', window=12, smoothing=3)
    
    # Visualiser les cycles
    fig7 = cycle_analyzer.plot_cycles()
    plt.show()
    
    # Analyser la performance par cycle
    cycle_analysis = backtest_engine.analyze_performance_by_economic_cycle(cycles)
    print("Performance par cycle économique:")
    display(cycle_analysis)
else:
    print("Une clé API FRED est nécessaire pour l'analyse des cycles économiques.")

In [ ]:
# Identifier les régimes de la courbe des taux
if '10Y' in treasury_yields.columns and '3M' in treasury_yields.columns:
    if api_key:
        yield_regimes = cycle_analyzer.identify_yield_curve_regime(treasury_yields)
        
        # Visualiser les régimes
        plt.figure(figsize=(14, 6))
        for regime in yield_regimes['regime'].unique():
            mask = yield_regimes['regime'] == regime
            plt.fill_between(
                yield_regimes[mask].index, 
                0, 1, 
                where=mask.values, 
                alpha=0.3, 
                label=regime
            )
            
        plt.plot(treasury_yields.index, treasury_yields['10Y_3M_Spread'], 'k-', label='10Y-3M Spread')
        plt.axhline(y=0, color='r', linestyle='--', alpha=0.5)
        plt.title('Régimes de la Courbe des Taux')
        plt.xlabel('Date')
        plt.ylabel('Spread (%)')
        plt.legend()
        plt.grid(True)
        plt.show()
        
        # Analyse combinée des cycles et de la courbe des taux
        combined_analysis = cycle_analyzer.generate_combined_analysis(etf_data, treasury_yields)
        
        # Afficher les recommandations
        recommendations = pd.DataFrame.from_dict(combined_analysis['recommendations'], orient='index')
        print("Recommandations de stratégie par cycle et régime:")
        display(recommendations)
        
        # Afficher les recommandations actuelles
        current_cycle = combined_analysis['current_cycle']
        current_regime = combined_analysis['current_regime']
        
        print(f"\nCycle économique actuel: {current_cycle}")
        print(f"Régime de courbe des taux actuel: {current_regime}")
        
        current_combo = f"{current_cycle}_{current_regime}"
        if current_combo in combined_analysis['recommendations']:
            rec = combined_analysis['recommendations'][current_combo]
            print("\nRecommandation de stratégie actuelle:")
            print(f"  Type: {rec['strategy_type']}")
            print(f"  Justification: {rec['rationale']}")
            print(f"  Meilleur ETF: {rec['best_etf']}")
            print(f"  Rendement attendu: {rec['expected_return']:.2f}%")
            print(f"  Volatilité attendue: {rec['expected_volatility']:.2f}%")
            print(f"  Ratio de Sharpe: {rec['sharpe_ratio']:.2f}")
    else:
        print("Une clé API FRED est nécessaire pour l'analyse des régimes de la courbe des taux.")

## 7. Analyse des Périodes de Hausse et de Baisse des Taux

In [ ]:
# Identifier les périodes de hausse et de baisse des taux
if '10Y' in treasury_yields.columns:
    # Calculer la variation du taux à 10 ans sur 3 mois
    treasury_yields['10Y_change_3m'] = treasury_yields['10Y'].diff(60)  # Environ 60 jours de trading
    
    # Définir les périodes de hausse et de baisse
    rate_regimes = pd.DataFrame(index=treasury_yields.index)
    rate_regimes['regime'] = 'stable'
    rate_regimes.loc[treasury_yields['10Y_change_3m'] > 0.5, 'regime'] = 'rising'  # Hausse significative
    rate_regimes.loc[treasury_yields['10Y_change_3m'] < -0.5, 'regime'] = 'falling'  # Baisse significative
    
    # Visualiser les régimes de taux
    plt.figure(figsize=(14, 8))
    plt.subplot(2, 1, 1)
    plt.plot(treasury_yields.index, treasury_yields['10Y'], label='Taux à 10 ans')
    plt.title('Rendement des Bons du Trésor à 10 ans')
    plt.ylabel('Rendement (%)')
    plt.grid(True)
    plt.legend()
    
    plt.subplot(2, 1, 2)
    for regime in rate_regimes['regime'].unique():
        mask = rate_regimes['regime'] == regime
        plt.fill_between(
            rate_regimes[mask].index, 
            0, 1, 
            where=mask.values, 
            alpha=0.3, 
            label=regime
        )
        
    plt.plot(treasury_yields.index, treasury_yields['10Y_change_3m'], 'k-', label='Variation sur 3 mois')
    plt.axhline(y=0, color='r', linestyle='--', alpha=0.5)
    plt.title('Régimes de Taux d\'Intérêt')
    plt.xlabel('Date')
    plt.ylabel('Variation (%)')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()
    
    # Analyser la performance par régime de taux
    # Faire correspondre les dates des régimes avec celles des portefeuilles
    rate_regimes_daily = rate_regimes.resample('D').ffill()
    
    # Créer un DataFrame avec les performances par régime
    performance_by_regime = pd.DataFrame(index=['rising', 'stable', 'falling'], 
                                         columns=list(portfolios.keys()) + ['Benchmark'])
    
    # Analyser chaque stratégie
    for name, portfolio in portfolios.items():
        returns = portfolio['returns'].dropna()
        for regime in ['rising', 'stable', 'falling']:
            # Filtrer les rendements correspondant au régime
            mask = rate_regimes_daily.loc[returns.index]['regime'] == regime
            regime_returns = returns[mask]
            
            if len(regime_returns) > 0:
                # Calculer le rendement annualisé
                annualized_return = regime_returns.mean() * 252 * 100  # En pourcentage
                performance_by_regime.loc[regime, name] = f"{annualized_return:.2f}%"
    
    # Ajouter le benchmark
    benchmark_returns = benchmark_data['returns'].dropna()
    for regime in ['rising', 'stable', 'falling']:
        mask = rate_regimes_daily.loc[benchmark_returns.index]['regime'] == regime
        regime_returns = benchmark_returns[mask]
        
        if len(regime_returns) > 0:
            annualized_return = regime_returns.mean() * 252 * 100  # En pourcentage
            performance_by_regime.loc[regime, 'Benchmark'] = f"{annualized_return:.2f}%"
    
    print("Performance par régime de taux d'intérêt:")
    display(performance_by_regime)

## 8. Conclusions et Recommandations

### Résumé des Performances

Notre analyse des différentes stratégies de portefeuille obligataire nous permet de tirer les conclusions suivantes :

1. **Performance Globale** :
   - La stratégie ___ a offert le meilleur rendement total sur la période.
   - La stratégie ___ a présenté le meilleur rapport rendement/risque (ratio de Sharpe).
   - La stratégie ___ a montré la plus faible volatilité.

2. **Performance par Cycle Économique** :
   - En phase d'expansion : la stratégie ___ est la plus performante.
   - En phase de pic : la stratégie ___ est la plus performante.
   - En phase de contraction : la stratégie ___ est la plus performante.
   - En phase de creux : la stratégie ___ est la plus performante.

3. **Performance par Régime de Taux** :
   - En période de hausse des taux : la stratégie ___ est la plus performante.
   - En période de taux stables : la stratégie ___ est la plus performante.
   - En période de baisse des taux : la stratégie ___ est la plus performante.

### Recommandations

Sur la base de notre analyse, voici nos recommandations pour la gestion d'un portefeuille obligataire :

1. **Stratégie de Base** : Utiliser une approche ___ comme fondement du portefeuille obligataire pour sa robustesse dans différentes conditions de marché.

2. **Ajustements Tactiques** :
   - Dans un environnement de hausse des taux, privilégier ___.
   - Dans un environnement de baisse des taux, privilégier ___.
   - Lors de l'inversion de la courbe des taux, privilégier ___.

3. **Allocation Stratégique** : Une combinaison des différentes approches peut offrir une meilleure diversification. Nous recommandons :
   - ___% en Laddering pour ___
   - ___% en Barbell pour ___
   - ___% en Bullet pour ___

En fonction de votre perspective actuelle sur le cycle économique et les taux d'intérêt, vous pouvez ajuster ces allocations pour optimiser la performance de votre portefeuille obligataire.